## 機械学習モデルをデバッグする
中古車の価格を予測する回帰モデルを Random Forest を用いて構築し、Responsible AI Toolbox でデバッグします。

### アジェンダ
0. 事前設定
1. ライブラリのインポート
2. データ準備
3. モデル学習
4. Responsible AI Dashboard 構築

<br>

### 事前設定
- Jupyter Kernel :  `raitoolbox` を選択する。
    - [setup.ipynb](../setup.ipynb) の手順に Python 環境のセットアップを完了しておくこと。

<br>

### 1. ライブラリのインポート
データの前処理やモデル学習に必要な Python ライブラリをインポートします。

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import pandas as pd

### 2. データ準備
自動車の価格に関するサンプルデータを Pandas DataFrame としてインポートします。

In [ ]:
df = pd.read_csv("../data/automobile.csv")
df = df.dropna()
target_feature = 'price'
data_df = df.drop([target_feature], axis=1)

In [ ]:
df.head()

In [ ]:
# 学習データとテストデータに分割
X_train_original, X_test_original, y_train, y_test = train_test_split(data_df, df[target_feature], test_size=0.5 , random_state=1234)

train_data = X_train_original.copy()
test_data = X_test_original.copy()
train_data[target_feature] = y_train
test_data[target_feature] = y_test

<br>

### 3. モデル構築

#### scikit learn パイプラインの作成

In [ ]:
def clean_data(X, y, target_feature):
    features = X.columns.values.tolist()
    pipe_cfg = {
        'num_cols': X.dtypes[(X.dtypes == 'int64') | (X.dtypes == "float64")].index.values.tolist(),
        'cat_cols': X.dtypes[X.dtypes == 'object'].index.values.tolist(),
    }
    num_pipe = Pipeline([
        ('num_imputer', SimpleImputer(strategy='median')),
        ('num_scaler', StandardScaler())
    ])
    cat_pipe = Pipeline([
        ('cat_imputer', SimpleImputer(strategy='constant', fill_value='?')),
        ('cat_encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))
    ])
    feat_pipe = ColumnTransformer([
        ('num_pipe', num_pipe, pipe_cfg['num_cols']),
        ('cat_pipe', cat_pipe, pipe_cfg['cat_cols'])
    ])
    X = feat_pipe.fit_transform(X)
    print("categorical:", pipe_cfg['cat_cols'])
    print("numerical:", pipe_cfg['num_cols'])

    return X, feat_pipe, features, pipe_cfg

X_train, feat_pipe, features, pipe_cfg = clean_data(X_train_original, y_train, target_feature)

Random Forest を用いたモデル学習を実行します。　

In [ ]:
model = RandomForestRegressor()
model.fit(X_train, y_train)

<br>

### 4. Responsible AI Dashboard 構築

Responsible AI Toolbox に関連するライブラリをインポートします。

In [ ]:
from raiwidgets import ResponsibleAIDashboard
from responsibleai import RAIInsights

In [ ]:
# scikit-learn Pipeline の指定
dashboard_pipeline = Pipeline(steps=[('preprocess', feat_pipe), ('model', model)])

# データや目的変数などの情報
rai_insights = RAIInsights(dashboard_pipeline, train_data, test_data, target_feature, 'regression',
                               categorical_features=pipe_cfg['cat_cols'])

In [ ]:
# モデル説明性 (InterpretML)
rai_insights.explainer.add()
# モデル誤差解析 (Error Analysis)
rai_insights.error_analysis.add()

インプットした情報をベースに計算をします。

In [ ]:
rai_insights.compute()

Responsible AI Toolbox ダッシュボードを生成します。表示される URI にブラウザからアクセスします。

In [ ]:
ResponsibleAIDashboard(rai_insights, locale="ja")